In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
print('**importing pakages**')
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
from string import digits

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import KFold, cross_val_score
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC 


print('**Loading data**')
df = pd.read_csv('./reddit_train.csv')

print('**Preprocessing data**')
df['clean_comments'] = df['comments'].replace(r'http\S', '', regex=True).replace(r'www\S', '', regex=True)
df['clean_comments'] = df['clean_comments'].str.replace("[^0-9a-zA-Z#+_]", " ")
df['clean_comments'] = df['clean_comments'].str.replace("&(\w);", " ")
#df['clean_comments'] = df['clean_comments'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
tokenized_comments = df['clean_comments'].apply(lambda x: x.split())

stemmer = PorterStemmer()
tokenized_comments = tokenized_comments.apply(lambda x: [stemmer.stem(word)for word in x])

lem = WordNetLemmatizer()
tokenized_comments = tokenized_comments.apply(lambda x: [lem.lemmatize(word) for word in x])
for i in range(len(tokenized_comments)):
    tokenized_comments[i] = ' '.join(tokenized_comments[i])

df['clean_comments'] = tokenized_comments


# count_vect = CountVectorizer( lowercase=True, preprocessor=None, tokenizer=None,
#                              stop_words='english',  ngram_range=(1,2),
#                              analyzer='word', max_df=0.5, min_df=3, max_features=20000, vocabulary=None,
#                              binary=True)
# count = count_vect.fit_transform(df['clean_comments'])
# train_data = count
tfidf_vectorizer = TfidfVectorizer(max_df=0.99, min_df=2, smooth_idf = True, norm='l2', ngram_range=(1,1), stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(df['clean_comments'])
train_data = tfidf


LE = LabelEncoder()
df['label'] = LE.fit_transform(df['subreddits'])
train_label = df['label'].to_numpy()

print('**Spliting traing and validation data**')

X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size = 0.2, random_state = 42)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alloc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alloc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**importing pakages**
**Loading data**
**Preprocessing data**
**Spliting traing and validation data**


In [2]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(95,), activation='relu', solver='adam',
                    alpha=0.00005, batch_size=200, learning_rate='invscaling',
                    learning_rate_init=0.0009, power_t=0.9, max_iter=4, shuffle=True,
                    random_state=False, tol=0.1, verbose=True, warm_start=False,
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                    validation_fraction=0.8, beta_1=0.9, beta_2=0.99, epsilon=1e-8,
                    n_iter_no_change=1)
clf = clf.fit(X_train, y_train)
predicted = clf.predict(X_valid)
np.mean(predicted == y_valid)

Iteration 1, loss = 2.62295746
Iteration 2, loss = 1.70363562
Iteration 3, loss = 1.30565989
Iteration 4, loss = 1.09853025


C:\Users\alloc\Anaconda3\envs\comp551a1\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.5860714285714286

In [ ]:
def kfold(clf,data,labels,k):
        i=0
        score=0
        batch_size=np.floor(data[:,1].size/k)
        kf = KFold(n_splits=k)
        kf.get_n_splits(train_data)
        for train_index, test_index in kf.split(data):
            training_data= data[train_index]
            
            training_label=labels[train_index]
            
            test_data= data[test_index]
            test_labels = labels[test_index]
            clf= clf.fit(training_data,training_label)
            preds = clf.predict(test_data)
            print("\nSegmentation #",i+1," Results:")
            print(np.mean(preds == test_labels))
            score+=np.mean(preds == test_labels)
            i+=1
            
        
        print("\nAverage Accuracy Score: ",score/k,"%")

In [ ]:
kfold(clf,train_data,train_label,10)

In [ ]:
print('**Loading data**')
df = pd.read_csv('./reddit_test.csv')

print('**Preprocessing data**')
df['clean_comments'] = df['comments'].replace('http\S', '', regex=True).replace('www\S', '', regex=True)
df['clean_comments'] = df['clean_comments'].str.replace("[^0-9a-zA-Z#+_]", " ")
df['clean_comments'] = df['clean_comments'].str.replace("&(\w);", " ")
#df['clean_comments'] = df['clean_comments'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
tokenized_comments = df['clean_comments'].apply(lambda x: x.split())

stemmer = PorterStemmer()
tokenized_comments = tokenized_comments.apply(lambda x: [stemmer.stem(word)for word in x])

lem = WordNetLemmatizer()
tokenized_comments = tokenized_comments.apply(lambda x: [lem.lemmatize(word) for word in x])
for i in range(len(tokenized_comments)):
    tokenized_comments[i] = ' '.join(tokenized_comments[i])

df['clean_comments'] = tokenized_comments



tfidf = tfidf_vectorizer.transform(df['clean_comments'])
test_data = tfidf


In [ ]:
print(test_data.shape)
clf = MLPClassifier(hidden_layer_sizes=(95,), activation='relu', solver='adam',
                    alpha=0.00005, batch_size=200, learning_rate='invscaling',
                    learning_rate_init=0.0009, power_t=0.9, max_iter=4, shuffle=True,
                    random_state=False, tol=0.1, verbose=True, warm_start=False,
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                    validation_fraction=0.8, beta_1=0.9, beta_2=0.99, epsilon=1e-8,
                    n_iter_no_change=1)
clf = clf.fit(train_data, train_label)

In [ ]:
res= clf.predict(test_data)
np.savetxt("predictions.csv", np.vstack((df['id'], LE.inverse_transform(res))).T, delimiter=",",fmt='%s')